# <font color="red">**STEP 1: DATA CURATION**</font>

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go

## **1. Google Trends**

In [79]:
gt_m_2011_2024 = pd.read_excel('./Data/Google trends/ALL_queries.xlsx', sheet_name='monthly', index_col=0)
gt_w_2019_2023 = pd.read_excel('./Data/Google trends/ALL_queries.xlsx', sheet_name='W 2019-2023', index_col=0)
gt_w_2019_2024 = pd.read_excel('./Data/Google trends/ALL_queries.xlsx', sheet_name='W 2019-2024feb', index_col=0)
gt_w_2023_2024 = pd.read_excel('./Data/Google trends/ALL_queries.xlsx', sheet_name='W 2023-2024', index_col=0)

gt_m_2011_2024 = pd.pivot_table(gt_m_2011_2024, index=gt_m_2011_2024.index, columns='query', values='value')
gt_m_2011_2024.index = gt_m_2011_2024.index + pd.offsets.MonthEnd(0)

gt_w_2019_2023 = pd.pivot_table(gt_w_2019_2023, index=gt_w_2019_2023.index, columns='query', values='value')
gt_w_2019_2024 = pd.pivot_table(gt_w_2019_2024, index=gt_w_2019_2024.index, columns='query', values='value')
gt_w_2023_2024 = pd.pivot_table(gt_w_2023_2024, index=gt_w_2023_2024.index, columns='query', values='value')

In [80]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_m_2011_2024.index, y=gt_m_2011_2024['Inflación'], mode='lines', name='Monthly 2011-2024'))
fig.add_trace(go.Scatter(x=gt_w_2019_2023.index, y=gt_w_2019_2023['Inflación'], mode='lines', name='Weekly 2019-2023'))
fig.add_trace(go.Scatter(x=gt_w_2019_2024.index, y=gt_w_2019_2024['Inflación'], mode='lines', name='Weekly 2019-2024'))
fig.add_trace(go.Scatter(x=gt_w_2023_2024.index, y=gt_w_2023_2024['Inflación'], mode='lines', name='Weekly 2023-2024'))

fig.update_layout(
    title='Inflation Trends',
    xaxis_title='Date',
    yaxis_title='Inflation',
    legend_title='Data Source'
)
fig.show()

<font color="cyan">Weekly-2019-2023 series are not the same scale as weekly-2023-2024 series. "aux" dataframe provides a rescale-factor to adjust weekly-2023-2024 series.</font> 

In [81]:
aux = ((gt_w_2019_2023.loc['2023-01-01':'2023-12-31', ]/gt_w_2023_2024.loc['2023-01-01':'2023-12-31', ])-1) + 1
aux = aux.mean(axis=0, skipna=True)
aux

query
Coste - Tema                                   NaN
Cuenta - Tema                             0.450000
Deflación - Tema                               NaN
Demanda - Economía                        0.953191
Dinero - Tema                                  NaN
Economía - Ciencia económica                   NaN
Gasto - Tema                                   NaN
Inflación                                 1.144476
Inflación - Tema                          0.550000
Interés - Tema                                 NaN
Macroeconomía - Campo de estudio               NaN
Mercado - Tema                                 NaN
Política - Tema                                NaN
Precio - Tema                                  NaN
Producto interno bruto - Tema             0.780000
Tasa - Matemáticas                        0.600000
canasta familiar                               inf
causas de la inflación                         NaN
como se calcula el ipc                         NaN
cpi                      

Using "aux", I adjust weekly-2023-2024 series as follows:

In [82]:
gt_w_2023_2024_adj = gt_w_2023_2024.copy()
for query in gt_w_2023_2024.columns:
    gt_w_2023_2024_adj[query] = gt_w_2023_2024[query] * aux[query]

In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_m_2011_2024.index, y=gt_m_2011_2024['Inflación'], mode='lines', name='Monthly 2011-2024'))
fig.add_trace(go.Scatter(x=gt_w_2019_2023.index, y=gt_w_2019_2023['Inflación'], mode='lines', name='Weekly 2019-2023'))
fig.add_trace(go.Scatter(x=gt_w_2019_2024.index, y=gt_w_2019_2024['Inflación'], mode='lines', name='Weekly 2019-2024'))
fig.add_trace(go.Scatter(x=gt_w_2023_2024_adj.index, y=gt_w_2023_2024_adj['Inflación'], mode='lines', name='Weekly 2023-2024'))

fig.update_layout(
    title='Inflation Trends',
    xaxis_title='Date',
    yaxis_title='Inflation',
    legend_title='Data Source'
)
fig.show()

Some GT series are mainly NaN (or Inf) values. "gt_words" is used to filter GT series with stable information.

In [84]:
gt_words = aux.copy()
gt_words = gt_words[~gt_words.isin([np.nan, np.inf, -np.inf])]
gt_words

query
Cuenta - Tema                             0.450000
Demanda - Economía                        0.953191
Inflación                                 1.144476
Inflación - Tema                          0.550000
Producto interno bruto - Tema             0.780000
Tasa - Matemáticas                        0.600000
deflactor                                 0.580000
demanda                                   1.151617
desempleo                                 0.641902
dinero                                    1.000000
economia                                  0.712863
el ipc                                    0.480000
indice de precios del consumidor          1.000000
ine                                       1.932609
ine bolivia                               0.847663
inflacion                                 0.840548
inflacion en bolivia                      1.020991
inflación argentina                       0.470000
inflación bolivia                         1.000000
ipc                      

### **1.1. Weekly Series**

"GT_weekly" dataframe comprises GT time series. However, some adjustments were applied.

In [85]:
all_weekly = pd.concat([gt_w_2019_2023, gt_w_2023_2024_adj], axis=0).filter(gt_words.index, axis=1)
all_weekly = all_weekly[~all_weekly.index.duplicated(keep='first')]
all_weekly = all_weekly.sort_index()
all_weekly

query,Cuenta - Tema,Demanda - Economía,Inflación,Inflación - Tema,Producto interno bruto - Tema,Tasa - Matemáticas,deflactor,demanda,desempleo,dinero,...,que es el ipc,que es inflacion,que es inflación,que es la inflacion,que es la inflación,que es pib,qué es inflación,qué es la inflación,tipo de inflación,Índice de precios al consumidor - Tema
date,,,,,,,,,,,,,,,,,,,,,
2018-12-30,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,18.000000,0.000000,74.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-06,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,35.000000,0.000000,77.0,...,63.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-13,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,51.000000,45.000000,72.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-20,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,43.000000,0.000000,73.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-27,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,43.000000,0.000000,73.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-01,0.0,0.0,30.900847,0.0,0.0,0.0,0.0,42.609844,26.959889,76.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-08,0.0,0.0,50.356935,0.0,0.0,0.0,0.0,40.306610,0.000000,81.0,...,0.0,0.0,0.0,68.575714,0.0,0.0,0.0,0.0,0.0,0.0
2024-12-15,0.0,0.0,25.178468,0.0,0.0,0.0,0.0,33.396905,0.000000,88.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Yet, some 0 values are showing up. 

In [86]:
zero_counts = (all_weekly == 0).sum()
print(zero_counts)

query
Cuenta - Tema                             306
Demanda - Economía                        303
Inflación                                  63
Inflación - Tema                          310
Producto interno bruto - Tema             312
Tasa - Matemáticas                        312
deflactor                                 310
demanda                                     0
desempleo                                  54
dinero                                      0
economia                                    0
el ipc                                    307
indice de precios del consumidor          313
ine                                         0
ine bolivia                                 3
inflacion                                   9
inflacion en bolivia                      229
inflación argentina                       310
inflación bolivia                         240
ipc                                       211
ipc bolivia                               299
ipc que es                  

"gt_words_2" is used to filter GT series with lower 0-values count.

In [87]:
gt_words_2 = zero_counts.copy()
gt_words_2 = gt_words_2[gt_words_2 < 250]
gt_words_2

query
Inflación                63
demanda                   0
desempleo                54
dinero                    0
economia                  0
ine                       0
ine bolivia               3
inflacion                 9
inflacion en bolivia    229
inflación bolivia       240
ipc                     211
la inflacion            130
la inflación            171
pib                       0
pib bolivia              12
que es inflacion        232
que es pib              138
dtype: int64

<font color="cyan">"GT_weekly" dataframe comprises final GT weekly time-series. </font>

In [88]:
GT_weekly = all_weekly.copy().filter(gt_words_2.index, axis=1)
GT_weekly.replace(0, np.nan, inplace=True)
GT_weekly.interpolate(method='linear', limit_direction="both", inplace=True)
GT_weekly = GT_weekly.round(0)
GT_weekly

query,Inflación,demanda,desempleo,dinero,economia,ine,ine bolivia,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
date,,,,,,,,,,,,,,,,,
2018-12-30,31.0,18.0,45.0,74.0,30.0,5.0,28.0,21.0,78.0,90.0,73.0,95.0,63.0,27.0,41.0,68.0,76.0
2019-01-06,31.0,35.0,45.0,77.0,49.0,22.0,28.0,21.0,78.0,90.0,73.0,95.0,63.0,40.0,44.0,68.0,76.0
2019-01-13,31.0,51.0,45.0,72.0,48.0,17.0,31.0,56.0,78.0,90.0,73.0,95.0,63.0,29.0,47.0,68.0,76.0
2019-01-20,31.0,43.0,41.0,73.0,49.0,13.0,28.0,48.0,78.0,90.0,73.0,95.0,63.0,38.0,29.0,68.0,76.0
2019-01-27,31.0,43.0,37.0,73.0,53.0,13.0,24.0,40.0,78.0,90.0,73.0,95.0,63.0,21.0,34.0,68.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-01,31.0,43.0,27.0,76.0,25.0,21.0,16.0,29.0,43.0,59.0,71.0,41.0,56.0,37.0,23.0,56.0,44.0
2024-12-08,50.0,40.0,27.0,81.0,23.0,21.0,24.0,32.0,43.0,59.0,71.0,41.0,45.0,30.0,23.0,56.0,44.0
2024-12-15,25.0,33.0,27.0,88.0,21.0,17.0,17.0,18.0,43.0,59.0,71.0,41.0,45.0,34.0,22.0,56.0,44.0


In [89]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_m_2011_2024.index, y=gt_m_2011_2024['Inflación'], mode='lines', name='Monthly 2011-2024'))
fig.add_trace(go.Scatter(x=all_weekly.index, y=all_weekly['Inflación'], mode='lines', name='Weekly'))
fig.add_trace(go.Scatter(x=GT_weekly.index, y=GT_weekly['Inflación'], mode='lines', name='Weekly Adjusted'))

fig.update_layout(
    title='Inflation Trends',
    xaxis_title='Date',
    yaxis_title='Inflation',
    legend_title='Data Source'
)
fig.show()

### **1.2. Monthly Series**

In [90]:
gt_month_1 = gt_m_2011_2024.loc[gt_m_2011_2024.index < '2019-03-31'].copy().filter(gt_words_2.index, axis=1).resample('M').mean()
gt_month_1 = gt_month_1.replace(0, np.nan).interpolate(method='linear', limit_direction="both")
gt_month_1

query,Inflación,demanda,desempleo,dinero,economia,ine,ine bolivia,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
date,,,,,,,,,,,,,,,,,
2011-01-31,42.0,22.0,42.0,41.0,60.0,43.0,54.0,58.0,59.0,100.0,79.0,44.0,33.0,50.0,29.0,100.0,54.0
2011-02-28,42.0,28.0,53.0,38.0,82.0,50.0,63.0,63.0,44.0,100.0,79.0,45.0,33.0,59.0,51.0,100.0,54.0
2011-03-31,44.5,41.0,44.0,46.0,99.0,59.0,79.0,82.0,78.0,100.0,54.0,57.0,33.0,92.0,77.0,100.0,54.0
2011-04-30,47.0,54.0,76.0,43.0,85.0,54.0,72.0,100.0,90.0,100.0,93.0,100.0,33.0,98.0,77.0,100.0,59.0
2011-05-31,29.0,55.0,62.0,47.0,83.0,57.0,75.0,75.0,68.0,100.0,88.0,73.0,33.0,78.0,69.0,89.0,56.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-31,27.0,89.0,41.0,75.0,47.0,16.0,16.0,48.0,29.0,28.0,49.0,39.0,29.0,73.0,60.0,35.0,47.0
2018-11-30,37.0,46.0,54.0,74.0,40.0,15.0,15.0,43.0,27.0,29.0,34.0,35.0,33.0,58.0,41.0,43.0,33.0
2018-12-31,18.0,26.0,27.0,75.0,31.0,10.0,9.0,21.0,11.0,29.0,22.0,17.0,25.0,47.0,36.0,36.0,32.0


In [91]:
gt_month_2 = GT_weekly.loc[GT_weekly.index >= '2019-03-31'].copy().resample('M').mean()
gt_month_2

query,Inflación,demanda,desempleo,dinero,economia,ine,ine bolivia,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
date,,,,,,,,,,,,,,,,,
2019-03-31,30.00,78.00,66.00,70.00,90.00,23.00,36.00,53.00,78.00,83.0,90.00,52.00,62.00,62.00,54.00,68.00,67.00
2019-04-30,28.25,76.00,44.75,64.50,76.75,22.25,34.25,51.75,78.00,80.5,82.25,54.50,61.50,63.00,53.25,69.00,61.25
2019-05-31,28.00,76.75,59.00,65.25,82.75,23.75,33.25,50.00,81.25,76.5,86.00,81.25,60.50,67.75,50.75,69.50,69.50
2019-06-30,26.60,68.80,56.60,67.00,68.20,21.80,36.00,47.00,93.00,73.0,92.00,72.00,59.40,66.00,60.20,70.20,60.20
2019-07-31,22.25,48.50,44.00,69.25,54.75,14.75,25.00,32.75,92.25,72.0,98.75,74.00,58.25,43.25,34.75,71.00,54.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,68.75,56.25,40.00,74.25,36.25,32.75,35.50,42.00,39.75,70.0,55.00,69.50,111.25,51.75,39.25,68.50,42.50
2024-09-30,32.20,61.20,40.80,71.40,31.80,33.20,31.20,28.80,53.40,49.2,58.60,59.60,59.20,49.60,39.00,64.20,38.20
2024-10-31,39.75,58.25,31.00,69.25,27.25,29.00,23.75,27.75,93.25,51.5,55.00,35.75,57.00,42.75,29.50,60.50,39.00


"GT_monthly" combines original monthly GT trends from 2011 to 2019(Feb) and the monthly averages computed on (final) weekly GT trends.  

In [92]:
GT_monthly = pd.concat([gt_month_1, gt_month_2], axis=0)
GT_monthly = GT_monthly.interpolate(method='linear', limit_direction="both").astype(int)
GT_monthly

query,Inflación,demanda,desempleo,dinero,economia,ine,ine bolivia,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
date,,,,,,,,,,,,,,,,,
2011-01-31,42,22,42,41,60,43,54,58,59,100,79,44,33,50,29,100,54
2011-02-28,42,28,53,38,82,50,63,63,44,100,79,45,33,59,51,100,54
2011-03-31,44,41,44,46,99,59,79,82,78,100,54,57,33,92,77,100,54
2011-04-30,47,54,76,43,85,54,72,100,90,100,93,100,33,98,77,100,59
2011-05-31,29,55,62,47,83,57,75,75,68,100,88,73,33,78,69,89,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,68,56,40,74,36,32,35,42,39,70,55,69,111,51,39,68,42
2024-09-30,32,61,40,71,31,33,31,28,53,49,58,59,59,49,39,64,38
2024-10-31,39,58,31,69,27,29,23,27,93,51,55,35,57,42,29,60,39


In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gt_m_2011_2024.index, y=gt_m_2011_2024['Inflación'], mode='lines', name='Monthly 2011-2024'))
fig.add_trace(go.Scatter(x=GT_monthly.index, y=GT_monthly['Inflación'], mode='lines', name='Monthly (Avg. 2019-2024)'))
fig.add_trace(go.Scatter(x=GT_weekly.index, y=GT_weekly['Inflación'], mode='lines', name='Weekly Adjusted'))

fig.update_layout(
    title='Inflation Trends',
    xaxis_title='Date',
    yaxis_title='Inflation',
    legend_title='Data Source'
)
fig.show()

## **2. Wholesale Prices**

The Agro-Environmental and Productive Observatory (OAP) is an agency under the Ministry of Rural Development and Lands in Bolivia. The OAP publishes daily bulletins of wholesale prices of selected agricultural products. It also publishes weekly bulletins on the behavior of wholesale prices and the supply of agricultural products. Time-series for domestic wholesale prices are sourced from teh OAP.

In [94]:
ws_products = pd.read_excel('./Data/Wholesale prices/DAILY_WHOLESALE_PRICES(adjusted).xlsx', sheet_name='dict')
ws_products["Description"].unique()

array(['Carne de res en gancho',
       'Carne de pollo entero evicerado NACIONAL',
       'Papa Desiree (Mediana)', 'Papa Huaycha o Imilla (Mediana)',
       'Yuca (Mediano - Grande)', 'Banana Cavendish (Mediana)',
       'Limón Sutil (Mediano) NACIONAL', 'Naranja criolla',
       'Papaya Salvietti (Mediana) NACIONAL', 'Piña', 'Platano', 'Sandia',
       'Manzana Red Delicious Mapleado (Mediana) IMPORTADO', 'Toronja',
       'Naranja injerto', 'Tomate', 'Aji en Vaina ', 'Arveja Verde ',
       'Cebolla Cabeza Roja ', 'Cebolla Cabeza IMPORTADA', 'Haba Verde',
       'Locoto', 'Maiz Choclo ', 'Pimenton ', 'Vainita', 'Zapallo ',
       'Zanahoria ', 'Arroz de primera', 'Arroz económico',
       'Arroz de segunda', 'Arroz de primera IMPORTADO',
       'Aceite vegetal envasado NACIONAL',
       'Aceite vegetal envasado IMPORTADO', 'Azúcar blanco NACIONAL',
       'Fideo NACIONAL', 'Harina blanca NACIONAL',
       'Harina blanca IMPORTADA', ' Leche fluida ', 'Leche en polvo ',
       'Mante

In [96]:
ws_raw = pd.read_excel('./Data/Wholesale prices/DAILY_WHOLESALE_PRICES(adjusted).xlsx', sheet_name='prices', index_col=0)
ws_raw = ws_raw.loc[ws_raw.index <= '2024-10-24'].ffill().bfill()
ws_raw

,beef_lp,beef_cb,beef_sc,beef_or,beef_po,beef_su,beef_tj,beef_tr,beef_co,beef_bol,...,wheat_lp,wheat_cb,wheat_sc,wheat_or,wheat_po,wheat_su,wheat_tj,wheat_tr,wheat_co,wheat_bol
2011-01-01,19.00,21.50,18.65,22.0,24.0,19.0,20.0,15.50,17.0,21.75,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,114.93
2011-01-03,19.00,21.50,18.65,21.5,24.0,19.0,20.0,15.50,17.0,20.00,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,114.93
2011-01-04,19.50,20.00,18.65,21.5,24.0,19.0,20.0,15.50,17.0,20.00,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,114.93
2011-01-05,19.00,20.50,18.25,21.5,24.0,19.0,20.0,15.50,17.0,20.00,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,114.93
2011-01-06,19.00,20.50,18.25,21.5,22.0,19.0,21.0,15.50,17.0,20.50,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,114.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-19,28.52,23.00,24.00,25.0,24.5,28.5,21.0,17.83,23.0,28.50,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.00
2024-10-21,28.52,23.00,24.75,25.0,24.5,28.5,21.0,17.83,23.0,28.50,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.00
2024-10-22,30.00,23.00,24.75,25.0,24.5,29.5,21.0,17.83,23.0,29.50,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.00
2024-10-23,30.00,23.00,24.00,25.0,24.5,29.5,21.0,17.83,23.0,29.50,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.00


Recalculate national median prices.

In [97]:
ws_bol_vars = ["beef_bol", "chicken_bol", "papa1_bol", "papa2_bol", "yuca_bol", "banana_bol", "lemon_bol", "orange_bol", "papaya_bol", "pineapple_bol", "platano_bol", "watermelon_bol", 
                "apple_bol", "grapefruit_bol", "orange2_bol", "tomato_bol", "redpepper_bol", "peas_bol", "onion_bol", "onion2_bol", "bean_bol", "chili_bol", "corn_bol", 
                "paprika_bol", "greenbean_bol", "squash_bol", "carrot_bol", "rice_bol", "rice2_bol", "rice3_bol", "rice4_bol", "oil_bol", "oil2_bol", "sugar_bol",
                "noodle_bol", "flour_bol", "flour2_bol", "milk_bol", "milk2_bol", "lard_bol", "veglard_bol", "ycorn_bol", "quinoa_bol", "sorghum_bol", "soy_bol", "wheat_bol"]

# Recalculate the national median prices
ws_adj = ws_raw.copy()

for column in ws_bol_vars:
    ws_adj[column] = ws_adj.iloc[:, (ws_adj.columns.get_loc(column) - 9):(ws_adj.columns.get_loc(column) ) ].median(axis=1)

ws_adj

,beef_lp,beef_cb,beef_sc,beef_or,beef_po,beef_su,beef_tj,beef_tr,beef_co,beef_bol,...,wheat_lp,wheat_cb,wheat_sc,wheat_or,wheat_po,wheat_su,wheat_tj,wheat_tr,wheat_co,wheat_bol
2011-01-01,19.00,21.50,18.65,22.0,24.0,19.0,20.0,15.50,17.0,19.0,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,168.0
2011-01-03,19.00,21.50,18.65,21.5,24.0,19.0,20.0,15.50,17.0,19.0,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,168.0
2011-01-04,19.50,20.00,18.65,21.5,24.0,19.0,20.0,15.50,17.0,19.5,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,168.0
2011-01-05,19.00,20.50,18.25,21.5,24.0,19.0,20.0,15.50,17.0,19.0,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,168.0
2011-01-06,19.00,20.50,18.25,21.5,22.0,19.0,21.0,15.50,17.0,19.0,...,280.0,168.0,114.93,250.0,150.0,145.0,345.0,NaN,NaN,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-19,28.52,23.00,24.00,25.0,24.5,28.5,21.0,17.83,23.0,24.0,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.0
2024-10-21,28.52,23.00,24.75,25.0,24.5,28.5,21.0,17.83,23.0,24.5,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.0
2024-10-22,30.00,23.00,24.75,25.0,24.5,29.5,21.0,17.83,23.0,24.5,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.0
2024-10-23,30.00,23.00,24.00,25.0,24.5,29.5,21.0,17.83,23.0,24.0,...,230.0,275.0,280.00,240.0,240.0,220.0,290.0,NaN,NaN,240.0


In [99]:
ws_adj = ws_adj.round(2)

In [100]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ws_raw.index, y=ws_raw['beef_bol'], mode='lines', name='Original'))
fig.add_trace(go.Scatter(x=ws_adj.index, y=ws_adj['beef_bol'], mode='lines', name='Adjusted'))

In [101]:
nan_counts = ws_adj.isna().sum()
nan_counts

beef_lp         0
beef_cb         0
beef_sc         0
beef_or         0
beef_po         0
             ... 
wheat_su        0
wheat_tj        0
wheat_tr     4070
wheat_co     4070
wheat_bol       0
Length: 460, dtype: int64

In [102]:
WS_weekly = ws_adj.resample('W').mean()
WS_monthly = ws_adj.resample('M').mean()

In [103]:
WS_monthly = WS_monthly.interpolate(method='linear', limit_direction="both")
WS_weekly = WS_weekly.interpolate(method='linear', limit_direction="both")

In [104]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ws_adj.index, y=ws_adj['tomato_bol'], mode='lines', name='Daily'))
fig.add_trace(go.Scatter(x=WS_weekly.index, y=WS_weekly['tomato_bol'], mode='lines', name='Weekly'))
fig.add_trace(go.Scatter(x=WS_monthly.index, y=WS_monthly['tomato_bol'], mode='lines', name='Monthly'))

In [105]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ws_adj.index, y=ws_adj['wheat_bol'], mode='lines', name='Daily'))
fig.add_trace(go.Scatter(x=WS_weekly.index, y=WS_weekly['wheat_bol'], mode='lines', name='Weekly'))
fig.add_trace(go.Scatter(x=WS_monthly.index, y=WS_monthly['wheat_bol'], mode='lines', name='Monthly'))

## **3. Commodity Prices**

International commodity prices are sourced from different international organizations and plataforms that provide daily reports of these prices.

In [106]:
COM_daily = pd.read_excel('./Data/Serie diaria de precios internacionales.xlsx', index_col=0, sheet_name='data')
COM_daily.replace(0, np.nan, inplace=True)
COM_daily = COM_daily.interpolate(method='linear', limit_direction="both")
COM_daily

,wti,gas_ny,gold,silver,zinc,tin,soybean,soy_flour,soy_oil,lead,copper,libor
2009-01-02,46.34,5.40,875.400,12.370000,1280.0,11625.0,328.823000,274.398000,835.033500,1090.0,3231.0,1.75250
2009-01-05,48.81,5.82,859.480,12.511136,1300.0,11500.0,336.826475,284.145691,828.799364,1120.0,3190.0,1.79375
2009-01-06,48.58,6.11,863.900,12.520000,1320.0,11950.0,335.895450,282.663000,862.596000,1178.0,3390.0,1.77000
2009-01-07,42.63,5.89,842.980,13.040000,1290.0,11600.0,331.762200,277.704000,872.298000,1140.0,3340.0,1.75000
2009-01-08,41.70,5.98,857.400,13.335000,1235.0,11400.0,331.578500,273.296000,872.298000,1149.0,3195.0,1.68625
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-28,83.17,1.54,2229.870,24.648800,2439.0,27451.0,438.124500,373.578000,1051.123500,2055.0,8867.0,5.64607
2024-03-29,83.44,1.54,2240.655,24.963100,2439.0,27451.0,437.757100,372.145400,1057.297500,2055.0,8867.0,5.64607
2024-04-01,83.71,1.63,2251.440,24.963100,2439.0,27451.0,437.757100,372.145400,1057.297500,2055.0,8867.0,5.64607
2024-04-02,85.15,1.63,2280.670,24.560786,2479.5,27897.0,435.460850,371.237562,1047.196500,2020.5,8991.0,5.64607


In [107]:
COM_monthly = COM_daily.resample('M').mean()
COM_weekly = COM_daily.resample('W').mean()

In [108]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=COM_monthly.index, y=COM_monthly['wti'], name='Monthly'))
fig.add_trace(go.Scatter(x=COM_weekly.index, y=COM_weekly['wti'], name='Weekly'))
fig.add_trace(go.Scatter(x=COM_daily.index, y=COM_daily['wti'], name='Daily'))
fig.update_layout(
    title='WTI Price',
    xaxis_title='Date',
    yaxis_title='Price'
)
fig.show()

## **4. Financial variables**

The Housing Development Unit (UFV) is a daily index, calculated based on inflation. It serves as a reference for financial transactions, contracts and all types of legal acts in national currency with the maintenance of value with respect to the evolution of domestic prices. This information is publicly available and published by the Central Bank of Bolivia.

The variable "exchange" is a daily time-series of the exchage rate USD/BOB that generates Google Finance, which is not neccesarily the as the exchange rate time-series published by the Central Bank of Bolivia; the latter shows a fixed echange rate since 2011. Conversely, Google Finance's USD/BOB exchage rate shows more variation that is better sutted for forecasting.

In [109]:
UFVFX_daily = pd.read_excel('./Data/UFV_FX_diario.xlsx', index_col=0)
UFVFX_daily = UFVFX_daily.loc['2011-01-01':'2024-03-22', ['ufv', 'dolarbo_comp']]
UFVFX_daily.interpolate(method='linear', limit_direction="both", inplace=True)
UFVFX_daily.rename(columns={'dolarbo_comp': 'exchange'}, inplace=True)
UFVFX_daily

,ufv,exchange
2011-01-01,1.56474,6.94
2011-01-02,1.56497,6.94
2011-01-03,1.56520,6.94
2011-01-04,1.56543,6.99
2011-01-05,1.56566,6.94
...,...,...
2024-03-18,2.48501,8.12
2024-03-19,2.48518,8.12
2024-03-20,2.48535,8.12
2024-03-21,2.48552,8.12


In [110]:
UFVFX_monthly = UFVFX_daily.copy().resample('M').mean()
UFVFX_weekly = UFVFX_daily.copy().resample('W').mean()

In [111]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=UFVFX_monthly.index, y=UFVFX_monthly['exchange'], name='Monthly'))
fig.add_trace(go.Scatter(x=UFVFX_weekly.index, y=UFVFX_weekly['exchange'], name='Weekly'))
fig.add_trace(go.Scatter(x=UFVFX_daily.index, y=UFVFX_daily['exchange'], name='Daily'))
fig.update_layout(
    title='USD/BOB',
    xaxis_title='Date',
    yaxis_title='Price'
)
fig.show()

## **5. CPI**

In [125]:
cpi_raw = pd.read_excel('./Data/CPI_1990_2024.xlsx', index_col=0)
cpi_raw.index = cpi_raw.index + pd.offsets.MonthEnd(0)

cpi_raw["lag_1"] = cpi_raw['ipc_all'].shift(1)
cpi_raw["lag_2"] = cpi_raw['ipc_all'].shift(2)
cpi_raw["lag_3"] = cpi_raw['ipc_all'].shift(3)
cpi_raw["lag_6"] = cpi_raw['ipc_all'].shift(6)
cpi_raw["lag_9"] = cpi_raw['ipc_all'].shift(9)
cpi_raw["lag_12"] = cpi_raw['ipc_all'].shift(12)

cpi_raw

,ipc_all,ipc_ali,ipc_vesti,ipc_viv,ipc_mueb,ipc_salud,ipc_recre,ipc_edu,ipc_bsss,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12
1990-01-31,17.412983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-28,17.396814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.412983,NaN,NaN,NaN,NaN,NaN
1990-03-31,17.457505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.396814,17.412983,NaN,NaN,NaN,NaN
1990-04-30,17.514421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.457505,17.396814,17.412983,NaN,NaN,NaN
1990-05-31,17.611392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.514421,17.457505,17.396814,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,116.245314,121.865398,103.115955,109.136327,119.950484,124.863721,113.653885,131.490908,116.209581,114.434010,113.895030,113.288776,111.433348,110.425657,110.506839
2024-09-30,117.267773,122.726749,104.604081,109.327853,122.683916,125.987008,114.524685,131.520899,119.536903,116.245314,114.434010,113.895030,111.941559,111.123491,110.440281
2024-10-31,119.196383,127.141329,105.877346,109.644520,124.040735,127.632275,114.917434,131.650495,121.567710,117.267773,116.245314,114.434010,112.579661,111.211060,110.429431
2024-11-30,120.922820,130.436377,106.442670,110.156938,125.231257,128.621443,115.371413,131.586245,123.184587,119.196383,117.267773,116.245314,113.288776,111.433348,110.425657


In [126]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cpi_raw.index, y=cpi_raw['ipc_all'], name='CPI'))

In [127]:
CPI_monthly = cpi_raw.filter(['ipc_all', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_9', 'lag_12'], axis=1)
CPI_monthly

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12
1990-01-31,17.412983,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-28,17.396814,17.412983,NaN,NaN,NaN,NaN,NaN
1990-03-31,17.457505,17.396814,17.412983,NaN,NaN,NaN,NaN
1990-04-30,17.514421,17.457505,17.396814,17.412983,NaN,NaN,NaN
1990-05-31,17.611392,17.514421,17.457505,17.396814,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-08-31,116.245314,114.434010,113.895030,113.288776,111.433348,110.425657,110.506839
2024-09-30,117.267773,116.245314,114.434010,113.895030,111.941559,111.123491,110.440281
2024-10-31,119.196383,117.267773,116.245314,114.434010,112.579661,111.211060,110.429431
2024-11-30,120.922820,119.196383,117.267773,116.245314,113.288776,111.433348,110.425657


## **6. Aggregated Dataset**

### **6.1. Monthly** 

In [128]:
merged_m = pd.merge(CPI_monthly, WS_monthly, left_index=True, right_index=True, how='inner')
merged_m = pd.merge(merged_m, UFVFX_monthly, left_index=True, right_index=True, how='inner')
merged_m = pd.merge(merged_m, COM_monthly, left_index=True, right_index=True, how='inner')
merged_m = pd.merge(merged_m, GT_monthly, left_index=True, right_index=True, how='inner')
merged_m

,ipc_all,lag_1,lag_2,lag_3,lag_6,lag_9,lag_12,beef_lp,beef_cb,beef_sc,...,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
2011-01-31,74.207255,73.260267,71.989803,71.196381,69.071086,68.561311,68.467691,18.492308,20.980769,18.450000,...,58,59,100,79,44,33,50,29,100,54
2011-02-28,75.439060,74.207255,73.260267,71.989803,69.800954,68.549203,68.581371,18.720833,20.500000,18.385000,...,63,44,100,79,45,33,59,51,100,54
2011-03-31,76.108818,75.439060,74.207255,73.260267,70.335479,68.646613,68.499278,19.166667,20.518519,18.140370,...,82,78,100,54,57,33,92,77,100,54
2011-04-30,76.125495,76.108818,75.439060,74.207255,71.196381,69.071086,68.561311,19.076923,19.153846,17.835769,...,100,90,100,93,100,33,98,77,100,59
2011-05-31,76.277495,76.125495,76.108818,75.439060,71.989803,69.800954,68.549203,19.000000,18.884615,17.827692,...,75,68,100,88,73,33,78,69,89,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,110.425657,110.429431,110.440281,110.506839,109.439785,108.697854,108.692935,23.211200,23.000000,21.000000,...,41,39,59,54,53,44,47,37,42,38
2023-12-31,111.123491,110.425657,110.429431,110.440281,109.678594,108.614602,108.818364,23.223333,23.000000,21.000000,...,31,39,68,67,55,51,35,24,54,44
2024-01-31,111.211060,111.123491,110.425657,110.429431,110.081702,108.814654,109.176930,24.120909,23.965909,20.954545,...,29,38,64,86,43,46,30,27,53,43
2024-02-29,111.433348,111.211060,111.123491,110.425657,110.506839,109.439785,108.697854,23.987391,24.956522,21.000000,...,31,38,58,66,38,38,37,34,51,41


In [129]:
M_isna_counts = merged_m.isna().sum()
M_isna_counts[M_isna_counts > 0]

quinoa_co     159
sorghum_tj    159
sorghum_co    159
soy_co        159
wheat_tr      159
wheat_co      159
dtype: int64

In [130]:
M_DATASET = merged_m.copy().dropna(axis=1)
M_DATASET.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 159 entries, 2011-01-31 to 2024-03-31
Columns: 492 entries, ipc_all to que es pib
dtypes: float64(475), int32(17)
memory usage: 601.8 KB


In [131]:
M_DATASET.isna().sum().sum()

0

In [132]:
M_DATASET.to_csv('./Data/M_DATASET.csv')

In [137]:
features_names = M_DATASET.columns[7:]
features_names

Index(['beef_lp', 'beef_cb', 'beef_sc', 'beef_or', 'beef_po', 'beef_su',
       'beef_tj', 'beef_tr', 'beef_co', 'beef_bol',
       ...
       'inflacion', 'inflacion en bolivia', 'inflación bolivia', 'ipc',
       'la inflacion', 'la inflación', 'pib', 'pib bolivia',
       'que es inflacion', 'que es pib'],
      dtype='object', length=485)

### **6.2. Weekly**

In [138]:
merged_w = pd.merge(UFVFX_weekly, WS_weekly, left_index=True, right_index=True, how='inner')
merged_w = pd.merge(merged_w, COM_weekly, left_index=True, right_index=True, how='inner')
merged_w = pd.merge(merged_w, GT_weekly, left_index=True, right_index=True, how='inner')

merged_w = merged_w[features_names]
merged_w

,beef_lp,beef_cb,beef_sc,beef_or,beef_po,beef_su,beef_tj,beef_tr,beef_co,beef_bol,...,inflacion,inflacion en bolivia,inflación bolivia,ipc,la inflacion,la inflación,pib,pib bolivia,que es inflacion,que es pib
2018-12-30,22.650000,21.000000,19.88,23.0,21.0,22.75,21.0,16.50,17.45,21.000000,...,21.0,78.0,90.0,73.0,95.0,63.0,27.0,41.0,68.0,76.0
2019-01-06,22.650000,20.800000,19.88,23.0,21.0,22.75,21.0,16.50,17.25,21.000000,...,21.0,78.0,90.0,73.0,95.0,63.0,40.0,44.0,68.0,76.0
2019-01-13,21.933333,20.500000,19.88,23.0,21.0,22.75,21.0,16.50,17.25,20.833333,...,56.0,78.0,90.0,73.0,95.0,63.0,29.0,47.0,68.0,76.0
2019-01-20,20.500000,20.833333,19.88,23.0,21.0,22.75,21.0,16.50,17.25,20.833333,...,48.0,78.0,90.0,73.0,95.0,63.0,38.0,29.0,68.0,76.0
2019-01-27,20.500000,21.000000,19.88,23.0,21.0,22.75,21.0,16.50,17.25,21.000000,...,40.0,78.0,90.0,73.0,95.0,63.0,21.0,34.0,68.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-25,23.550000,24.000000,21.00,24.0,24.0,25.50,21.0,18.83,23.00,23.550000,...,25.0,38.0,56.0,57.0,45.0,36.0,52.0,53.0,50.0,41.0
2024-03-03,23.183333,24.000000,21.00,24.0,24.0,25.50,21.0,18.83,23.00,23.183333,...,25.0,37.0,55.0,51.0,55.0,58.0,55.0,33.0,49.0,49.0
2024-03-10,22.713333,24.000000,21.00,24.0,24.0,25.50,21.0,18.83,23.00,23.000000,...,35.0,37.0,53.0,65.0,40.0,51.0,64.0,46.0,48.0,61.0
2024-03-17,22.570000,24.000000,21.00,24.0,24.0,25.50,21.0,18.83,23.00,23.000000,...,29.0,37.0,51.0,68.0,47.0,45.0,60.0,30.0,48.0,85.0


In [139]:
W_isna_counts = merged_w.isna().sum()
W_isna_counts[W_isna_counts > 0]

Series([], dtype: int64)

In [140]:
W_DATASET = merged_w.copy()
W_DATASET.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2018-12-30 to 2024-03-24
Columns: 485 entries, beef_lp to que es pib
dtypes: float64(485)
memory usage: 1.0 MB


In [141]:
W_DATASET.to_csv('./Data/W_DATASET.csv')